In [14]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from tqdm.auto import tqdm
import json

In [20]:
# === SETTINGS ===
video_path = "data/calibration.mp4"
chessboard_shape = (8, 8)  # chessboard (#columns, #rows)
frame_interval = 10       # process every 10th frame
output_file = "calibration_data.json"

In [21]:
chessboard_size = (chessboard_shape[0]-1, chessboard_shape[1]-1)  # number of inner corners per chessboard row and column

# Prepare object points like (0,0,0), (1,0,0), ..., (8,5,0)
objp = np.zeros((chessboard_size[0]*chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

objpoints = []  # 3D points in real world space
imgpoints = []  # 2D points in image plane

In [22]:
cap = cv2.VideoCapture(video_path)
frame_count = 0
success = True
total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("[INFO] Scanning video for chessboard patterns...")

# for i in tqdm(range(total_frame_count // frame_interval), desc="Processing frames", unit="frame"):
for i in tqdm(range(total_frame_count), desc="Processing frames", unit="frame"):
# while success:
    if not(success):
        print("[INFO] No more frames to process.")
        break

    success, frame = cap.read()
    if frame_count == 0:
        height, width = frame.shape[:2]
        image_size = (width, height)
    if not success:
        continue
    if frame_count % frame_interval != 0:
        frame_count += 1
        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        refined_corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                           criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        imgpoints.append(refined_corners)

        # Optional: draw and show corners
        # cv2.drawChessboardCorners(frame, chessboard_size, refined_corners, ret)

    frame_count += 1

    # for _ in range(frame_interval - 2):
    #     success, frame = cap.read()
    #     if not success:
    #         break
    #     frame_count += 1

cap.release()

[INFO] Scanning video for chessboard patterns...


Processing frames:   0%|          | 0/691 [00:00<?, ?frame/s]

In [23]:
# Perform calibration
if len(objpoints) >= 5:
    print("[INFO] Performing camera calibration...")

    flags = (cv2.CALIB_RATIONAL_MODEL | 
         cv2.CALIB_THIN_PRISM_MODEL |
         cv2.CALIB_TILTED_MODEL)

    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, image_size, None, None, flags=flags
)

    # ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    #     objpoints, imgpoints, gray.shape[::-1], None, None
    # )

    print("\n=== Calibration Results ===")
    print("Intrinsic Matrix:\n", camera_matrix.ravel())
    print("Distortion Coefficients:\n", dist_coeffs.ravel())

    data = {
        "distortion_coefficients": dist_coeffs.ravel().tolist(),
        "intrinsic_matrix": camera_matrix.ravel().tolist(),
    }
    with open(output_file, "w") as f:
        json.dump(data, f, indent=4)

    print(f"\n[INFO] Calibration data saved to: {output_file}")
else:
    print("[ERROR] Not enough valid chessboard frames for calibration.")

[INFO] Performing camera calibration...

=== Calibration Results ===
Intrinsic Matrix:
 [504.02353288   0.         426.06999938   0.         505.68393343
 240.31930792   0.           0.           1.        ]
Distortion Coefficients:
 [ 3.25253718e-02  2.45361644e-01  2.05374936e-03  1.83000823e-02
  2.31659992e+00 -7.82916462e-02  6.79446174e-01  1.91531877e+00
 -7.48709548e-03 -7.36026290e-03 -2.81460908e-03 -2.07595546e-02
 -2.23806677e-02 -2.17785516e-02]

[INFO] Calibration data saved to: calibration_data.json


In [26]:
total_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], camera_matrix, dist_coeffs)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    total_error += error

mean_error = total_error / len(objpoints)
print("Mean re-projection error:", mean_error)

Mean re-projection error: 0.022828473264944596
